# Welcome to Antidote

## Introduction

In this tutorial, you will be introduced to Antidote Databases. You can test and see how an eventually consistent database works. We will guide you through the CRDT data types supported by Antidote, transactions, and how Antidote resolves inconsistencies. For more information, see http://syncfree.github.io/antidote/.

> Note that this Jupyter notebook is based on Antidote Java API, but it is tailored for tuturial purposes. As a result, this tutorial might not reflect the full capabilities of Antidote. To find out how to use the API, please refer to the __[official Antidote Java API](https://www.javadoc.io/doc/eu.antidotedb/antidote-java-client/0.1.0)__.

## Antidote data types

Antidote supports several Conflict-free replicated data type (CRDT). Each type handles inconsistencies differently, and we will go through some of them in this section. The supported data types include:
- __Number types__
    - Counter
    - Fat counter
    - Integer
- __Registers__
    - Last-writer wins register
    - Multi-value register
- __Sets__
    - Add-wins set (Observed-remove set)
    - Remove-win set
- __Maps__
    - Grow-only map
    - Add-wins map
    - Remove-resets map

The first step is to make sure Antidote service is up and running.

In [1]:
init()

Antidote session created. Connected to Antidote node 1.

In Antidote, an object of any data types consists of 2 parts: a key and a value. You need a key in order to update/remove/reset/read your data object in an Antidote bucket. A bucket is an internal data structure used by Antidote where objects are stored. In this tutorial, we have 1 bucket which uses a unique session id as its key. An object key should be unique across the bucket, regardless of data types.

### Counter

Counter is an integer object with some operations. First, we need to create a key for our counter object. We assign "c1" as its key.

In [2]:
counter_key = getCounterKey("c1")

COUNTER_c1

The counter datatype is simply an integer which can be incremented or decremented. Let's try incrementing our c1 counter by 10, and decrementing it by 1.

In [3]:
applyUpdate(incrementCounter(counter_key, 10))
applyUpdate(incrementCounter(counter_key, -1))

Updated key 'COUNTER_c1' on Antidote 0

Reading a counter object returns the aggregated value from all received operations. In our example, our c1 should now equal to 9.

In [4]:
read(counter_key)

9

Fat counter and Integer is similar to Counter, except that, in Fat counter, you can reset the counter to its inital state, while in Integer, you can assign a value to it.

### Register

Register can store a single-valued object. We use a register object to store string in the following example.

In [5]:
register_key = getLWRegisterKey("r1")

LWWREG_r1

In [6]:
applyUpdate(assignLWRegister(register_key, "Hello"))

Updated key 'LWWREG_r1' on Antidote 0

In [7]:
read(register_key)

Hello

### Set

Set can store a collection of values of the same type. This notebook supports only a Set of String.

In [35]:
set_key = getSetKey("s1")

ORSET_s1

In [36]:
applyUpdate(addToSet(set_key, "A","B","C","D"))

Updated key 'ORSET_s1' on Antidote 0

In [37]:
read(set_key)

[A, B, C, D]

In [38]:
applyUpdate(removeFromSet(set_key, "C","D"))

Updated key 'ORSET_s1' on Antidote 0

In [39]:
read(set_key)

[A, B]

### Map

Map can store a collection of values of different Antidote data types. In the following example, we create a map which stores 2 element objects: an integer, and a fat counter object.

In [14]:
map_key = getAWMapKey("m1")

AWMAP_m1

In [16]:
integer_key = getIntegerKey("i1")

INTEGER_i1

In [19]:
fatcounter_key = getFatCounterKey("f1")

FATCOUNTER_f1

To update a map, we actually update element objects of the map.

In [20]:
applyUpdate(updateAWMap(map_key, assignInteger(integer_key, 1)
                               , incrementFatCounter(fatcounter_key, 2)))

Updated key 'AWMAP_m1' on Antidote 0

To read from a map, we also need to specify the key of the element object.

In [18]:
readFromMap(map_key, integer_key)

1

In [21]:
readFromMap(map_key, fatcounter_key)

2

In [40]:
applyUpdate(removeFromAWMap(map_key, integer_key))

Updated key 'AWMAP_m1' on Antidote 0

In [41]:
readFromMap(map_key, integer_key)

Cell returned null.

In [42]:
readFromMap(map_key, fatcounter_key)

2

## Transactions

So far, we use `applyUpdate()` to update the Antidote objects, but you can also group several updates to create an atomic transaction. Let's see an example below. We create a transaction that will assign a value to x and incremrent y.

In [49]:
x = getIntegerKey("x")

INTEGER_x

In [50]:
y = getCounterKey("y")

COUNTER_y

In [54]:
tx = startTransaction()

eu.antidotedb.client.InteractiveTransaction@30d0ff1a

In [55]:
addToTransaction(tx, incrementInteger(x, 10))

Added key 'INTEGER_x' to transaction

In [56]:
addToTransaction(tx, incrementCounter(y, 1))

Added key 'COUNTER_y' to transaction

The updates will take effect only after a transaction is committed.

In [57]:
commitTransaction(tx)

Transaction committed on Antidote 0

In [58]:
read(x)

20

In [59]:
read(y)

3

## Data replication

In Antidote, replicated databases are eventually consistent. Each data type resolves the consistencies differently. In this section, we will demonstrate this by using 2 replicated Antidote nodes: _Antidote1_ and _Antidote2_. By default, this notebook connects to Antidote1.

!!!!!!!!!!add image here!!!!!!

Both Antidote1 and Antidote2 are currently connected. Any updates to either one of them will be replicated to the other one.

In [60]:
x = getIntegerKey("x")

INTEGER_x

In [61]:
applyUpdate(assignInteger(x, 2))

Updated key 'INTEGER_x' on Antidote 0

In [62]:
read(x)

2

Now we switch to Antidote2 to see if x is 2 as well.

In [63]:
switchAntidote(2)

Connected to Antidote 2

In [64]:
read(x)

2

## Inconsistency resolution

In this section, we will see how Antidote resolves conflicts when updates are done on disconnected replicas.

### Last-writer-wins

The final value of the object is the value updated at the latest timestamp. The Last-writer-wins register uses this mechanism.

In [79]:
lww_key = getLWRegisterKey("lww1")
applyUpdate(assignLWRegister(lww_key, "original"))
read(lww_key)

original

In [72]:
disconnectAntidotes()

Disconnecting Antidote nodes.

In [73]:
switchAntidote(1)

Connected to Antidote 1

In [74]:
applyUpdate(assignLWRegister(lww_key, "update at 1"))

Updated key 'LWWREG_lww1' on Antidote 0

In [75]:
switchAntidote(2)

Connected to Antidote 2

In [76]:
applyUpdate(assignLWRegister(lww_key, "update at 2"))

Updated key 'LWWREG_lww1' on Antidote 0

In [77]:
connectAntidotes()

Connecting Antidote nodes.

In [78]:
read(lww_key)

update at 2